In [ ]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 35.2 MB/s eta 0:00:00
  Attempting uninstall: autograd
    Found existing installation: autograd 1.6.2
    Uninstalling autograd-1.6.2:
      Successfully uninstalled autograd-1.6.2


In [ ]:
import time
import os
import copy

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, transforms

# Pennylane
import pennylane as qml
from pennylane import numpy as np

torch.manual_seed(42)
np.random.seed(42)

# Plotting
import matplotlib.pyplot as plt

# OpenMP: number of parallel threads.
os.environ["OMP_NUM_THREADS"] = "1"

In [ ]:
n_qubits = 4                # Number of qubits
step = 0.0004               # Learning rate
batch_size = 8              # Number of samples for each training step
num_epochs = 3              # Number of training epochs
q_depth = 6                 # Depth of the quantum circuit (number of variational layers)
gamma_lr_scheduler = 0.1    # Learning rate reduction applied every 10 epochs.
q_delta = 0.01              # Initial spread of random quantum weights
start_time = time.time()    # Start of the computation timer

In [ ]:
dev = qml.device("default.qubit", wires=n_qubits)
dev

<DefaultQubit device (wires=4, shots=None) at 0x7d6fc2d5be50>

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [ ]:
df = pd.read_csv('acetylcholinesterase_06_bioactivity_data_3class_pIC50_pubchem_fp.csv')

In [ ]:
X = df.drop('pIC50', axis=1)
X

,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,PubchemFP9,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4615,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4616,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4617,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4618,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
Y = df.pIC50
Y

0       6.124938
1       6.999996
2       4.301030
3       6.522877
4       6.096909
          ...   
4615    6.943091
4616    9.677781
4617    6.562248
4618    4.118045
4619    4.259637
Name: pIC50, Length: 4620, dtype: float64

In [ ]:
print(X.shape)
print(Y.shape)


(4620, 881)
(4620,)


**Removing Low Variance features**

In [ ]:
from sklearn.feature_selection import VarianceThreshold
selection = VarianceThreshold(threshold=(.8 * (1 - .8)))
X = selection.fit_transform(X)

In [ ]:
X.shape

(4620, 140)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
print(X_train.shape)
print(X_test.shape)

(3696, 140)
(924, 140)


In [ ]:
Y_train[0:5]

3010    4.580044
2077    4.720000
2117    4.820448
1255    6.721244
3192    6.400007
Name: pIC50, dtype: float64

**Creating an QML**

In [ ]:
def H_layer(nqubits):
    """Layer of single-qubit Hadamard gates.
    """
    for idx in range(nqubits):
        qml.Hadamard(wires=idx)


def RY_layer(w):
    """Layer of parametrized qubit rotations around the y axis.
    """
    for idx, element in enumerate(w):
        qml.RY(element, wires=idx)


def entangling_layer(nqubits):
    """Layer of CNOTs followed by another shifted layer of CNOT.
    """
    # In other words it should apply something like :
    # CNOT  CNOT  CNOT  CNOT...  CNOT
    #   CNOT  CNOT  CNOT...  CNOT
    for i in range(0, nqubits - 1, 2):  # Loop over even indices: i=0,2,...N-2
        qml.CNOT(wires=[i, i + 1])
    for i in range(1, nqubits - 1, 2):  # Loop over odd indices:  i=1,3,...N-3
        qml.CNOT(wires=[i, i + 1])

In [ ]:
@qml.qnode(dev, interface="torch")
def quantum_net(q_input_features, q_weights_flat):
    """
    The variational quantum circuit.
    """

    # Reshape weights
    q_weights = q_weights_flat.reshape(q_depth, n_qubits)

    # Start from state |+> , unbiased w.r.t. |0> and |1>
    H_layer(n_qubits)

    # Embed features in the quantum node
    RY_layer(q_input_features)

    # Sequence of trainable variational layers
    for k in range(q_depth):
        entangling_layer(n_qubits)
        RY_layer(q_weights[k])

    # Expectation values in the Z basis
    exp_vals = [qml.expval(qml.PauliZ(position)) for position in range(n_qubits)]
    return tuple(exp_vals)

In [ ]:
class DressedQuantumNet(nn.Module):
    """
    Torch module implementing the *dressed* quantum net.
    """

    def __init__(self):
        """
        Definition of the *dressed* layout.
        """

        super().__init__()
        # self.pre_net1 = nn.Linear(512, n_qubits)
        self.q_params = nn.Parameter(q_delta * torch.randn(q_depth * n_qubits))
        self.post_net = nn.Linear(n_qubits, 1)

    def forward(self, input_features):
        """
        Defining how tensors are supposed to move through the *dressed* quantum
        net.
        """

        # obtain the input features for the quantum circuit
        # by reducing the feature dimension from 512 to 4
        pre_out = input_features
        # print(input_features.shape)
        q_in = torch.tanh(pre_out) * np.pi / 2.0

        # Apply the quantum circuit to each element of the batch and append to q_out
        q_out = torch.Tensor(0, n_qubits)
        q_out = q_out.to(device)
        for elem in q_in:
            q_out_elem = torch.hstack(quantum_net(elem, self.q_params)).float().unsqueeze(0)
            q_out = torch.cat((q_out, q_out_elem))

        # return the two-dimensional prediction from the postprocessing layer
        return self.post_net(q_out)

In [ ]:
model = nn.Sequential(
    nn.Linear(140, 128),
    nn.ReLU(),
    nn.Linear(128, 256),
    nn.ReLU(),
    nn.Linear(256, 4),
    DressedQuantumNet())

print(model)

Sequential(
  (0): Linear(in_features=140, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=256, bias=True)
  (3): ReLU()
  (4): Linear(in_features=256, out_features=4, bias=True)
  (5): DressedQuantumNet(
    (post_net): Linear(in_features=4, out_features=1, bias=True)
  )
)


In [ ]:
model = model.to(device)

In [ ]:
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# def evaluation(X,Y,model):
#   with torch.no_grad():
#     total,correct = 0,0
#     for i in range(0, len(X), batch_size):
#         Xbatch = torch.tensor(X[i:i+batch_size])
#         Xbatch = Xbatch.to(device)
#         Xbatch = Xbatch.to(torch.float32)
#         y_pred = model(Xbatch)
#         ybatch = torch.tensor(Y[i:i+batch_size].values)
#         ybatch = ybatch.to(device)
#         ybatch = ybatch.to(torch.float32)



#     for data in zip(X,Y):
#       inputs,labels = data
#       inputs = torch.tensor(inputs)
#       labels = torch.tensor(labels)
#       inputs,labels = inputs.to(device).to(torch.float32),labels.to(device).to(torch.float32)
#       outputs = model(inputs)
#       _,pred = torch.max(outputs.data,1)
#      # pred = pred + 1
#       total += labels.size(0)
#       correct += (pred==labels).sum().item()

#     return 100*correct/total

In [ ]:
from tqdm import tqdm
n_epochs = 100
batch_size = 10

for epoch in tqdm(range(n_epochs)):
    for i in range(0, len(X_train), batch_size):
        Xbatch = torch.tensor(X_train[i:i+batch_size])
        Xbatch = Xbatch.to(device)
        Xbatch = Xbatch.to(torch.float32)
        # print(Xbatch.dtype)
        # print(model[0].weight.dtype)
        y_pred = model(Xbatch)
        ybatch = torch.tensor(Y_train[i:i+batch_size].values)
        ybatch = ybatch.to(device)
        ybatch = ybatch.to(torch.float32)
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # train_acc = evaluation(X_train,Y_train,model)
    # test_acc = evaluation(X_test,Y_test,model)
    print(f'Finished epoch {epoch}, latest loss {loss} ')#, train acc {train_acc}, test_acc {test_acc}')


  0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([6])) that is different to the input size (torch.Size([6, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
  1%|          | 1/100 [02:59<4:55:39, 179.19s/it]

Finished epoch 0, latest loss 10.649503707885742 


  2%|▏         | 2/100 [05:55<4:50:03, 177.59s/it]

Finished epoch 1, latest loss 4.6823954582214355 


  3%|▎         | 3/100 [08:53<4:47:10, 177.64s/it]

...

Finished epoch 97, latest loss 2.7608649730682373 


 99%|█████████▉| 99/100 [4:57:36<03:07, 187.03s/it]

Finished epoch 98, latest loss 2.778519868850708 


100%|██████████| 100/100 [5:00:30<00:00, 180.30s/it]

Finished epoch 99, latest loss 2.7812740802764893 


In [ ]:
# Load wights file of the best model :
wights_file = '/content/Weights-025--1.39267.hdf5' # choose the best checkpoint
model.load_weights(wights_file) # load it

AttributeError: ignored

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

Y_test_pred = model.predict(X_test)

r2_score = r2_score(Y_test, Y_test_pred)
print("The accuracy of our model is {}%".format(round(r2_score, 2) *100))